In [ ]:
import cv2
import os
import math
import numpy as np
from vehicle import Vehicle

def get_centroid(x, y, w, h):
    return (x + w // 2, y + h // 2)

def distance_between_points(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

video = r"traffic_video.mp4"

if not os.path.exists(video):
    print(f"Error: No se encuentra el archivo {video}")
    # exit() 

print(f"Intentando abrir: {video}")
cap = cv2.VideoCapture(video)
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=25)

object_paths = {}  
next_car_id = 1
min_contour_area = 1100 
min_centroid_distance = 80  

descending_line_y = 600
car_count = 0
crossed_line_cars = set()
car_count2 = 0
crossed_line_cars2 = set()

ascending_line_y = 478
car_count3 = 0
crossed_line_cars3 = set()

# Conjunto para recordar qué coches sumaron doble y pintarlos de rojo
anomalous_cars = set()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Ha ocurrido un error al leer el vídeo o se ha acabado la transmisión.")
        break

    frame = cv2.resize(frame, (1280, 720))
    roi = frame[230:820, 0:1280]  
    descending_line_y_in_roi = descending_line_y - 230    
    ascending_line_y_in_roi = ascending_line_y - 230

    mask = bg_subtractor.apply(roi)
    _, mask = cv2.threshold(mask, 230, 255, cv2.THRESH_BINARY)
    mask = cv2.medianBlur(mask, 5)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:  
            x, y, w, h = cv2.boundingRect(contour)
            detections.append((x, y, w, h))
 
    overlay = roi.copy()

    # DIBUJO ROI SEMITRANSPARENTE
    #cv2.rectangle(overlay, (0,0), (roi.shape[1], roi.shape[0]), (0, 140, 255), -1)
    
    # DIBUJO POLÍGONOS
    # Polígono Verde
    pts_verde = np.array([[330, 0], [430, 0], [1050, 450], [850, 450]], np.int32)
    pts_verde = pts_verde.reshape((-1, 1, 2))
    cv2.fillPoly(overlay, [pts_verde], (0, 255, 0))
        
    # Polígono Azul
    pts_azul = np.array([[230, 0], [330, 0], [600, 450], [350, 450]], np.int32)
    pts_azul = pts_azul.reshape((-1, 1, 2))
    cv2.fillPoly(overlay, [pts_azul], (255, 0, 0))

    # Polígono Amarillo
    pts_amarillo = np.array([[420, 0], [520, 0],[1200, 290], [960, 290]], np.int32)
    pts_amarillo = pts_amarillo.reshape((-1, 1, 2))
    cv2.fillPoly(overlay, [pts_amarillo], (0, 255, 255))

    # Polígono zona roja (ramal de incorporación a la vía)
    pts_rojo = np.array([[500, 0], [1200, 20],[1200, 100], [750, 50]], np.int32)
    pts_rojo = pts_rojo.reshape((-1, 1, 2))
    
    alpha = 0.2
    cv2.addWeighted(overlay, alpha, roi, 1 - alpha, 0, roi)

    # Líneas de conteo
    cv2.line(roi, (300, descending_line_y_in_roi), (550, descending_line_y_in_roi), (255, 0, 0), 2)
    cv2.line(roi, (720, descending_line_y_in_roi), (950, descending_line_y_in_roi), (0, 255, 0), 2)
    cv2.line(roi, (860, ascending_line_y_in_roi), (1120, ascending_line_y_in_roi), (0, 255, 255), 2) 

    updated_paths = {}
    
    for (x, y, w, h) in detections:
        centroid = get_centroid(x, y, w, h)
        found = False
            
        # Variable para rastrear el ID actual y decidir el color después
        current_id = None 

        # Comparamos con cada Vehiculo conocido
        for car_id, vehicle in object_paths.items():
            last_centroid = vehicle.last_centroid()
            dist = distance_between_points(centroid, last_centroid)

            if dist < min_centroid_distance: 
                found = True
                current_id = car_id # Capturamos el ID encontrado
                vehicle.update(centroid, bbox=(x, y, w, h))
                updated_paths[car_id] = vehicle

                if len(vehicle.path) >= 2:
                    prev_x, prev_y = vehicle.prev_centroid()
                    curr_x, curr_y = vehicle.last_centroid()

                    # Lógica de conteo carriles descendentes
                    if (prev_y < descending_line_y_in_roi <= curr_y):

                        # Lógica de conteo carril azul
                        if (200 <= curr_x <= 600 and 
                            car_id not in crossed_line_cars):
                            car_count += 1
                            crossed_line_cars.add(car_id)  

                        # Lógica de conteo carril verde
                        if (680 <= curr_x <= 950 and
                            car_id not in crossed_line_cars2):
                            
                            # CONDICIÓN ANÓMALA 1 (sumar +2 al contador)
                            if (850 <= curr_x <= 868):
                                car_count2 += 2
                                crossed_line_cars2.add(car_id)
                                anomalous_cars.add(car_id) # Marcar como anómalo
                            else:
                                car_count2 += 1
                                crossed_line_cars2.add(car_id) 

                    # Lógica de conteo carril amarillo
                    elif (prev_y > ascending_line_y_in_roi >= curr_y):
                        if (860 <= curr_x <= 1260 and
                            car_id not in crossed_line_cars3):

                            # CONDICIÓN ANÓMALA 2 (sumar +2 al contador)
                            if (929 <= curr_x <= 935):
                                car_count3 += 2
                                crossed_line_cars3.add(car_id)
                                anomalous_cars.add(car_id) # Marcar como anómalo
                            else:
                                car_count3 += 1
                                crossed_line_cars3.add(car_id)

                break
                    
        if not found:
            v = Vehicle(next_car_id, centroid, bbox=(x, y, w, h))
            updated_paths[next_car_id] = v
            current_id = next_car_id # El ID es el nuevo generado
            next_car_id += 1

        # Lógica para el color del rectángulo
        color_rectangulo = (255, 140, 255) # Lila por defecto

        # 1. Comprobamos si está en el carril de incoporación
        dist_poly = cv2.pointPolygonTest(pts_rojo, (float(centroid[0]), float(centroid[1])), False)
        if dist_poly >= 0:
            color_rectangulo = (0, 0, 255) # Rojo por zona

        # 2. Comprobamos si es un vehículo anómalo (cuenta x2)
        # Esto sobrescribe o mantiene el rojo si el coche ha sido marcado en anomalous_cars
        if current_id in anomalous_cars:
            color_rectangulo = (0, 0, 255) # Rojo por condición anómala

        # Dibujamos con el color determinado
        cv2.rectangle(roi, (x, y), (x + w, y + h), color_rectangulo, 2)
        cv2.circle(roi, centroid, 4, (0, 0, 255), -1)

    object_paths = updated_paths

    cv2.rectangle(frame, (105, 20), (410, 120), (190, 180, 160), -1)
    cv2.putText(frame, f"Conteo coches: {car_count}", (110, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Conteo coches: {car_count2}", (110, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Conteo coches: {car_count3}", (110, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow("Deteccion", frame)

    if cv2.waitKey(10) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

Intentando abrir: traffic_video2.mp4
